# Results

In [42]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


from tensorflow.keras import models
from tensorflow.keras.layers import LSTM,Dense

In [50]:
df_securities = pd.read_csv('./data/securities.csv')
df_prices = pd.read_csv('./data/prices-split-adjusted.csv')
df_prices.fillna(float(0))

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [72]:
class Company:
    def __init__(self, symbol, name, df):
        self.symbol = symbol
        self.name = name
        self.df = df
        self.model = None
        
        self.train_data = []
        self.test_data = []
        
        self.x = []
        self.y = []
        
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
    def prep_data(self, scaler, split):
        scaled_df = scaler.fit_transform(np.array(self.df).reshape(-1,1)).copy()
        size = int(len(self.df)*split)

        self.train_data = scaled_df[0:size,:]
        self.test_data = scaled_df[size:,:]
        
    
    def build_data(self, step):
        x, y = [], []
        for i in range(int(len(self.train_data)-step-1)):
            x.append(self.train_data[i:int(i+step),0])
            y.append(self.train_data[int(i+step),0])
            
        X_train = x
        X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))  
        self.X_train = X_train
        self.y_train = y

        x, y = [], []
        for i in range(int(len(self.test_data)-step-1)):
            x.append(self.test_data[i:int(i+step),0])
            y.append(self.test_data[int(i+step),0])
            
            
        X_test = x
        X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)      
        self.X_test = X_test
        self.y_test = y    
        
        
        
        


def get_companies(num):
    choices = []
    names = []
    while(len(choices) < 5):
        i = random.randint(0,df_securities.shape[0]-1)
        symbol = df_securities['Ticker symbol'].iloc[i]
        name = df_securities['Security'].iloc[i]
        temp_df = df_prices[df_prices['symbol']==symbol]
        if (temp_df.shape[0] > 1500) and (symbol not in choices):
            choices.append(symbol)
            names.append(name)
        
    return choices, names

def get_multiple_df(symbols, names):
    directory = []
    
    for symbol in symbols:
        single_df = df_prices[df_prices['symbol']==symbol]
        single_df = df_prices.drop(columns=['symbol'])
        single_df = df_prices.set_index('date')
        single_df = df_prices['close'].to_frame()
        
        directory.append(single_df)
    return directory


def fix_data(data, scaler):
    return scaler.inverse_transform(np.array(data).reshape(-1,1))


def build_model(optimizer, step):
    model = models.Sequential()

    model.add(LSTM(64,return_sequences=True,input_shape=(step,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(32,return_sequences=True))
    model.add(LSTM(50,return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    model.summary()
    
    return model

In [76]:
directory_size = 5
directory = []

split = 0.7
step = 60

scalar = sklearn.preprocessing.MinMaxScaler()

symbols, names = get_companies(directory_size)
selected_dfs = get_multiple_df(symbols)
for i in range(directory_size):
    directory.append(Company(symbols[i], names[i], selected_dfs[i]))
    
len(directory)

5

In [77]:
for company in directory:
    company.prep_data(scalar, split)
    company.build_data(split)
    print("X Training Shape: "+str(X_train.shape))
    print("X Test Shape: "+str(X_test.shape))
    print("Y Training Shape: "+str(y_train.shape))
    print("Y Test Shape: "+str(y_test.shape))

In [78]:

print("X Training Shape: "+str(X_train.shape))
print("X Test Shape: "+str(X_test.shape))
print("Y Training Shape: "+str(y_train.shape))
print("Y Test Shape: "+str(y_test.shape))

NameError: name 'scaler' is not defined